# Langgraph RAG

## Lib import

In [ ]:
import os
import logging
import operator
from typing import List, Literal, Annotated, Optional, Union, Any
from typing_extensions import TypedDict
import chromadb
from chromadb.config import Settings
from pydantic import BaseModel, Field, validator

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnablePassthrough,
    RunnableSerializable,
)
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver

## Init config and API key

In [ ]:
#
# key
#
from dotenv import load_dotenv

load_dotenv()

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
linkup_api_key = os.getenv("LINKUP_API_KEY")

In [ ]:
#
# config
#
import tomllib


def load_config(config_file):
    try:
        with open(config_file, "rb") as f:
            config = tomllib.load(f)
            return config
    except Exception as e:
        print(f"Load config file error: {e}")
        return None


# load config file
deepseek_llm_model = None
silicon_base_url = None
silicon_llm_model = None
huggingface_embed_model = None

config_data = load_config("../config/config.toml")
if config_data:
    log_level = config_data.get("log_level")
    if log_level:
        logging.basicConfig(level=log_level)

    # deepseek
    deepseek_llm_model = config_data.get("deepseek", {}).get("model")
    deepseek_llm_temperature = config_data.get("deepseek", {}).get("temperature")
    deepseek_llm_max_tokens = config_data.get("deepseek", {}).get("max_tokens")

    # silicon
    silicon_base_url = config_data.get("silicon", {}).get("base_url")
    silicon_llm_model = config_data.get("silicon", {}).get("model")

    # huggingface
    huggingface_embed_model = config_data.get("huggingface", {}).get("embed_model")


# deepseek
deepseek_llm_model = deepseek_llm_model or "deepseek-chat"

# silicon
silicon_base_url = silicon_base_url or "https://api.siliconflow.cn/v1"
silicon_llm_model = silicon_llm_model or "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = (
    huggingface_embed_model or "sentence-transformers/all-MiniLM-L6-v2"
)

## LLM Init

In [ ]:
# init LLM mod
llm_deepseek = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=deepseek_llm_temperature or 0.3,
    max_tokens=deepseek_llm_max_tokens,
    timeout=None,
    top_p=0.9,
    frequency_penalty=0.7,
    presence_penalty=0.5,
    max_retries=3,
)

## LLM functions

### Question Re-writer

## RAG Graph

In [ ]:
################################################################################
### RAG state
################################################################################
class RagState(TypedDict):
    question: str
    answer: str
    retrieves: List
    web_searchs: List
    #documents: Annotated[list, operator.add]

In [ ]:
################################################################################
### RAG nodes
################################################################################
def node_rewrite_qestion(state: RagState):
    question = state["question"]

    # Re-write question
    # better_question = question_rewriter.invoke({"question": question})

    return {"question": better_question or question}


def node_retrieve(state: RagState):
    question = state["question"]
    retrieves = []

    # Retrieval
    # retrieves = retriever.invoke(question)
    return {"retrieves": retrieves}


def node_retrieve_grade(state: RagState):
    question = state["question"]
    retrieves = state["retrieves"]

    # Score each doc
    filtered_docs = []
    for d in retrieves:
        # grade = retrieval_grader.invoke(
        #    {"question": question, "document": d.page_content}
        # )

        if grade is None:
            grade = {"score": 5}

        score = grade.score
        if score >= 5:
            d.metadata["relevance_score"] = score
            filtered_docs.append(d)

    return {"retrieves": filtered_docs}

def node_web_search(state: RagState):
    question = state["question"]

    # Web search
    docs = []
    # docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"web_searchs": web_results}

def node_generate(state: RagState):
    question = state["question"]
    retrieves = state["retrieves"]
    web_searchs = state["web_searchs"]

    # generation
    generation = ""

    return {"answer": generation}

In [ ]:
################################################################################
### Edges conditional functions
################################################################################
def condition_retrieve(state: RagState):
    question = state["question"]
    retrieves = state["retrieves"]

    if retrieves:
        return "success"
    else:
        return "failure"


def condition_retrieve_grade(state: RagState):
    question = state["question"]
    retrieves = state["retrieves"]

    if not retrieves:
        return "irrelevant"
    
    for d in retrieves:
        if not d.metadata["relevance_score"]: continue
        if d.metadata["relevance_score"] >= 7:
            return "relevant"
       
    return "somewhat"

In [ ]:
def build_rag_graph():
    ################################################################################
    ### Create Graph
    ################################################################################
    workflow = StateGraph(RagState)

    ################################################################################
    ### Add nodes
    ################################################################################
    workflow.add_node("rewrite_qestion", node_rewrite_qestion)
    workflow.add_node("retrieve", node_retrieve)
    workflow.add_node("retrieve_grade", node_retrieve_grade)
    workflow.add_node("web_search", node_web_search)
    workflow.add_node("generate", node_generate)

    ################################################################################
    ### Add edges
    ################################################################################
    ## rewrite qestion
    workflow.add_edge(START, "rewrite_qestion")
    workflow.add_edge("rewrite_qestion", "retrieve")

    ## retrieve
    workflow.add_conditional_edges(
        "retrieve",
        condition_retrieve,
        {
            "success": "retrieve_grade",
            "failure": "web_search",
        },
    )
    workflow.add_conditional_edges(
        "retrieve_grade",
        condition_retrieve_grade,
        {
            "relevant": "generate",
            "somewhat": "web_search",
            "irrelevant": "web_search",
        },
    )

    ## web_search
    workflow.add_edge("web_search", "generate")

    ## generate
    workflow.add_edge("generate", END)

    return workflow.compile()

In [ ]:

################################################################################
### Build RAG Graph
################################################################################
rag_graph = build_rag_graph()

In [ ]:
from IPython.display import Image, display

display(Image(rag_graph.get_graph().draw_mermaid_png()))

## Test